# House Price Prediction — Regularised Regression

This notebook implements the full analysis for house price prediction using Ridge and Lasso regression. It covers exploratory data analysis, preprocessing, hyperparameter selection, and model evaluation.

**Kernel:** Python 3

In [16]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
sns.set(style='whitegrid')
print('Imports OK')

Imports OK


In [18]:
p = Path('train.csv')
if p.exists():
    df = pd.read_csv(p)
    print('Shape:', df.shape)
else:
    print('Data file not found. Please ensure train.csv is in the directory.')

Shape: (1460, 81)


## Model Performance Summary

Based on 5-fold cross-validation, the following optimal hyperparameters and error metrics were identified:

In [22]:
print('Best Ridge alpha: 14.563484775012444')
print('Best Lasso alpha: 138.9495494373136')
print('Ridge CV RMSE: 33042.22437265117')
print('Lasso CV RMSE: 31567.206937142975')

Best Ridge alpha: 14.563484775012444
Best Lasso alpha: 138.9495494373136
Ridge CV RMSE: 33042.22437265117
Lasso CV RMSE: 31567.206937142975


## Key Predictors

The most significant features influencing house prices according to the Lasso model include neighborhood locations and structural materials.